# Percentage from ATL comparison

In this exercise, we will fetch top 250 cryptocurrencies by market cap from www.coingecko.com and compare their percentage change from their all time low price. This comparison data is good for figuring out what might be a good price to invest in for these cryptocurrencies.

# Library Imports

First we will import the list of open source libraries that we will need for this exercise.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter

# Importing List of Top 250 cryptocurrencies

Next, we will import the list of Top 250 cryptocurrencies as on https://www.coingecko.com/.

The cryptocurrencies in this list may change over time so the ideal way to get the list of top 100 cryptocurrencies is to connect to the Coingecko API and get the real time data from their API database. The API documentation can be found here: https://www.coingecko.com/en/api/documentation.

Since this is a one time exercise, we will be using the free API version, there is no need to get an API key now. However, if we need to access the database frequently and choose a paid plan, an API key will be required. We will be using the /coins/markets API end point to get the list of top 250 cryptocurrencies. The end point also includes other relevant data as well, such as, ticker, price, market capitalization, atl, ath, etc which we will need later on.

Let's import the list to our Jupyter notebook using the following commands:

In [ ]:
api_url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=250&page=1&sparkline=false"
data = requests.get(api_url).json()
print(data)

# Creating a Pandas DataFrame and parsing our API call

Next we will create a pandas dataframe where we can store the above data. We will call our dataframe "atl_dataframe".

In [ ]:
my_columns = ['Name', 'Ticker','Current Price', 'Market Capitalization', 'ATH', 'ATH Date', 'ATH Change %', 'ATL', 'ATL Date', 'ATL Change %']
atl_dataframe = pd.DataFrame(columns = my_columns)
atl_dataframe

In [ ]:
for i in data:
    name = i['name']
    ticker = i['symbol']
    price = i['current_price']
    market_cap = i['market_cap']
    ath = i['ath']
    ath_date = i['ath_date']
    ath_change = i['ath_change_percentage']
    atl = i['atl']
    atl_date = i['atl_date']
    atl_change = i['atl_change_percentage']
    atl_dataframe = atl_dataframe.append(
    pd.Series(
        [
            name,
            ticker,
            price,
            market_cap,
            ath,
            ath_date,
            ath_change,
            atl,
            atl_date,
            atl_change,
            
        ],
            index = my_columns,
    ),
        ignore_index = True
)
atl_dataframe

# Output the result in an Excel file

Finally, we will output the above dataframe result in an excel file using xlsxwriter.

In [ ]:
writer = pd.ExcelWriter('atl_comparison.xlsx', engine='xlsxwriter')
atl_dataframe.to_excel(writer, sheet_name='ATL Comparison', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Name', string_template],
                    'B': ['Ticker', string_template],
                    'C': ['Current Price', dollar_template],
                    'D': ['Market Capitalization', dollar_template],
                    'E': ['ATH', dollar_template],
                    'F': ['ATH Date', string_template],
                    'G': ['ATH Change %', percent_template],
                    'H': ['ATL', dollar_template],
                    'I': ['ATL Date', string_template],
                    'J': ['ATL Change %', percent_template]
                    }

for column in column_formats.keys():
    writer.sheets['ATL Comparison'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['ATL Comparison'].write(f'{column}1', column_formats[column][0], string_template)

In [ ]:
writer.save()